In [14]:
# install the pycuda library
!pip install pycuda
!pip install numpy
# if installed then ignore. 
# device initialization, memory cleanup and context creation
import pycuda.autoinit
# functions for memory handling, as allocation, deallocation and transfers, etc.
import pycuda.driver as cuda

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
'''version 1: atomic'''
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule

# Define the size of the matrices
M = 1024
N = 1024
K = 1024

# Generate two random matrices on the host
a = np.random.randn(M, K).astype(np.float32)
b = np.random.randn(K, N).astype(np.float32)

# Allocate memory on the GPU
a_gpu = drv.mem_alloc(a.nbytes)
b_gpu = drv.mem_alloc(b.nbytes)
c_gpu = drv.mem_alloc(M * N * 4)

# Copy the matrices from the host to the GPU
drv.memcpy_htod(a_gpu, a)
drv.memcpy_htod(b_gpu, b)

# Define the CUDA kernel function
mod = SourceModule("""
  __global__ void matmul_kernel(float *c, float *a, float *b, int M, int N, int K)
  {
    const int i = threadIdx.y + blockDim.y * blockIdx.y;
    const int j = threadIdx.x + blockDim.x * blockIdx.x;

    if (i < M && j < N) {
        float sum = 0.0f;
        __shared__ float partial_sums[16][16];
        for (int k = 0; k < K; k += 16) {
            partial_sums[threadIdx.y][threadIdx.x] = 0.0f;
            __syncthreads();
            for (int kk = 0; kk < 16; kk++) {
                partial_sums[threadIdx.y][threadIdx.x] += a[i * K + k + kk] * b[(k + kk) * N + j];
            }
            __syncthreads();
            if (threadIdx.x < 8 && threadIdx.y < 8) {
                atomicAdd(&c[(i + threadIdx.y) * N + j + threadIdx.x], partial_sums[threadIdx.y][threadIdx.x] + partial_sums[threadIdx.y + 8][threadIdx.x] + partial_sums[threadIdx.y][threadIdx.x + 8] + partial_sums[threadIdx.y + 8][threadIdx.x + 8]);
            }
        }
    }
  }
""")

# Get the kernel function from the module
matmul_kernel = mod.get_function("matmul_kernel")

# Define the grid and block sizes
block_size = (16, 16, 1)
grid_size = ((N + block_size[0] - 1) // block_size[0], (M + block_size[1] - 1) // block_size[1], 1)

# Call the kernel function on the GPU
matmul_kernel(c_gpu, a_gpu, b_gpu, np.int32(M), np.int32(N), np.int32(K), block=block_size, grid=grid_size)

# Copy the result from the GPU to the host
c = np.empty((M, N), dtype=np.float32)
drv.memcpy_dtoh(c, c_gpu)

# Print the result
print(c)


[[-18.895441    0.         26.247822  ...   0.         53.828026
    0.       ]
 [  0.          0.          0.        ...   0.          0.
    0.       ]
 [ -7.7703266   0.         43.969017  ...   0.         10.106428
    0.       ]
 ...
 [  0.          0.          0.        ...   0.          0.
    0.       ]
 [ 54.831234    0.         23.400795  ...   0.        136.02373
    0.       ]
 [  0.          0.          0.        ...   0.          0.
    0.       ]]


In [16]:
'''version 2: naive'''
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule

# Define the size of the matrices
M = 1024
N = 1024
K = 1024

# Generate two random matrices on the host
a = np.random.randn(M, K).astype(np.float32)
b = np.random.randn(K, N).astype(np.float32)

# Allocate memory on the GPU
a_gpu = drv.mem_alloc(a.nbytes)
b_gpu = drv.mem_alloc(b.nbytes)
c_gpu = drv.mem_alloc(M * N * 4)

# Copy the matrices from the host to the GPU
drv.memcpy_htod(a_gpu, a)
drv.memcpy_htod(b_gpu, b)

# Define the CUDA kernel function
mod = SourceModule("""
  __global__ void matmul_kernel(float *c, float *a, float *b, int M, int N, int K)
  {
    const int i = threadIdx.y + blockDim.y * blockIdx.y;
    const int j = threadIdx.x + blockDim.x * blockIdx.x;

    if (i < M && j < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += a[i * K + k] * b[k * N + j];
        }
        c[i * N + j] = sum;
    }
  }
""")

# Get the kernel function from the module
matmul_kernel = mod.get_function("matmul_kernel")

# Define the grid and block sizes
block_size = (16, 16, 1)
grid_size = ((N + block_size[0] - 1) // block_size[0], (M + block_size[1] - 1) // block_size[1], 1)

# Call the kernel function on the GPU
matmul_kernel(c_gpu, a_gpu, b_gpu, np.int32(M), np.int32(N), np.int32(K), block=block_size, grid=grid_size)

# Copy the result from the GPU to the host
c = np.empty((M, N), dtype=np.float32)
drv.memcpy_dtoh(c, c_gpu)

# Print the result
print(c)

[[-51.903515   15.017205  -43.736603  ...  41.17424     6.2617483
  -25.255705 ]
 [ -3.7582579   4.838371   -4.937979  ...   7.4522486   7.221829
   -8.59482  ]
 [-22.765707  -73.2108    -38.719425  ... -35.58352    25.162642
  -40.991062 ]
 ...
 [ -5.3446245  -4.4299736  -3.3474386 ...  -9.698916   11.614197
  -47.058777 ]
 [ 11.108996   20.98836    68.949715  ...   3.5221226  16.153843
   -5.140696 ]
 [-39.756      41.226673  -17.758255  ...  31.898722    9.524585
   38.07006  ]]


In [17]:
'''atomics version '''
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule

# Define the size of the matrices and tile size
M = 1024
N = 1024
K = 1024
TILE_SIZE = 16

# Generate two random matrices
a = np.random.randn(M, K).astype(np.float32)
b = np.random.randn(K, N).astype(np.float32)

# Define the CUDA kernel function
mod = SourceModule("""
  #define TILE_SIZE %(tile_size)d

  __global__ void matmul(float *a, float *b, float *c, int m, int n, int k) {
    __shared__ float s_a[TILE_SIZE][TILE_SIZE];
    __shared__ float s_b[TILE_SIZE][TILE_SIZE];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * TILE_SIZE + ty;
    int col = bx * TILE_SIZE + tx;

    float sum = 0.0;
    for (int i = 0; i < (k-1)/TILE_SIZE+1; i++) {
      if (row < m && i*TILE_SIZE+tx < k) {
        s_a[ty][tx] = a[row*k + i*TILE_SIZE+tx];
      } else {
        s_a[ty][tx] = 0.0;
      }
      if (i*TILE_SIZE+ty < k && col < n) {
        s_b[ty][tx] = b[(i*TILE_SIZE+ty)*n + col];
      } else {
        s_b[ty][tx] = 0.0;
      }
      __syncthreads();

      for (int j = 0; j < TILE_SIZE; j++) {
        sum += s_a[ty][j] * s_b[j][tx];
      }
      __syncthreads();
    }

    // Hierarchical atomics
    for (unsigned int stride = TILE_SIZE/2; stride > 0; stride >>= 1) {
        __syncthreads();
        if (ty < stride && row < m && col < n) {
            unsigned int num_tiles = ((n-1)/TILE_SIZE+1)*((m-1)/TILE_SIZE+1);
            unsigned int tile_id = by*((n-1)/TILE_SIZE+1)+bx;
            unsigned int offset = tile_id * TILE_SIZE * TILE_SIZE;
            unsigned int mask = 0xffffffff >> (32 - __ffs(TILE_SIZE*TILE_SIZE));
            unsigned int* c_tile = (unsigned int*)c + offset;
            atomicAdd(&c_tile[(row % TILE_SIZE + ty) * TILE_SIZE + col % TILE_SIZE], __float_as_uint(sum) & mask);
        }
    }
  }
""" % {"tile_size": TILE_SIZE})


# Get the kernel function from the module
matmul = mod.get_function("matmul")

# Allocate memory on the GPU
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(M*N * np.dtype(np.float32).itemsize)

# Copy the matrices to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Launch the kernel function
grid = ((N-1)//TILE_SIZE+1, (M-1)//TILE_SIZE+1, 1)
block = (TILE_SIZE, TILE_SIZE, 1)
matmul(a_gpu, b_gpu, c_gpu, np.int32(M), np.int32(N), np.int32(K), block=block, grid=grid)

# Copy the result from the GPU to the CPU
c = np.empty((M, N), dtype=np.float32)
cuda.memcpy_dtoh(c, c_gpu)

# Print the result
print(c)


TypeError: not enough arguments for format string